Problem statement:

Situation: Energy is produced in different forms and consumed by the regions in different rates.   

Complication: Energy prices are increasing due to many reasons including but not limited to war, logistics and sanctions.   

Question: Is it possible to see what types of energy have been historically produced and consumed by the different regions? Is it possible to predict how the productions and consumption will look like in the future?   

Answer: ???


In [109]:
import pandas as pd
import numpy as np

import seaborn as sns
sns.set_theme(style='darkgrid', palette='crest')
import matplotlib.pyplot as plt


from sklearn.datasets import load_diabetes  # our diabetes dataset
from sklearn.model_selection import (
    train_test_split,
)  # will allow us to helpfully subdivide our data
from sklearn.linear_model import (
    LinearRegression,
)  # a simple model we can play around with

from sktime.datasets import load_airline
from sktime.utils.plotting import plot_series
from sktime.forecasting.naive import NaiveForecaster

from sktime.forecasting.base import ForecastingHorizon


from sktime.forecasting.theta import ThetaForecaster

from sktime.forecasting.arima import AutoARIMA


# pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)

In [110]:
leveranser_flytande = pd.read_table('../data/to-use/leveranser-flytande-bransle-region-ar.csv', sep=',')
elproduction_bransleanvandning = pd.read_table('../data/to-use/elproduction-bransleanvandning-region-produktionssatt-bransletyp-ar.csv', sep=',')
fjarrvarmeproduktion = pd.read_table('../data/to-use/fjarrvarmeproduktion-region-produktionssatt-bransletyp-ar.csv', sep=',')
slutanvandning = pd.read_table('../data/to-use/slutanvandning-region-forbrukarkategori-bransletyp-kategori-energityp-ar.csv', sep=',')
bransleforbrukning = pd.read_table('../data/to-use/branslefarbrukning-far-produktion-per-ar.csv', sep=',')
energidata = pd.read_table('../data/to-use/energidata-per-ar.csv', sep=',')

datasets = [leveranser_flytande, elproduction_bransleanvandning, 
            fjarrvarmeproduktion, slutanvandning, bransleforbrukning, energidata]


## FIll NAs
leveranser_flytande.m3 = leveranser_flytande.m3.fillna(method='ffill')
fjarrvarmeproduktion.MWh = fjarrvarmeproduktion.MWh.interpolate()
# slutanvandning = slutanvandning.fillna('fffill')
energidata.MWh = energidata.MWh.fillna(method='ffill')


# Slice dataset for regions
leveranser_flytande = leveranser_flytande[leveranser_flytande.region.isin(['0160 T�by', '0117 �ster�ker', '0180 Stockholm', '0980 Gotland', '01 Stockholms l�n', '22 V�sternorrlands l�n', '09 Gotlands l�n', '00 Riket', '2581 Pite�', '1480 G�teborg'])].reset_index(drop=True)
elproduction_bransleanvandning = elproduction_bransleanvandning[elproduction_bransleanvandning.region.isin(['0160 T�by', '0117 �ster�ker', '0180 Stockholm', '0980 Gotland', '01 Stockholms l�n', '22 V�sternorrlands l�n', '09 Gotlands l�n', '00 Riket', '2581 Pite�', '1480 G�teborg'])].reset_index(drop=True)
fjarrvarmeproduktion = fjarrvarmeproduktion[fjarrvarmeproduktion.region.isin(['0160 T�by', '0117 �ster�ker', '0180 Stockholm', '0980 Gotland', '01 Stockholms l�n', '22 V�sternorrlands l�n', '09 Gotlands l�n', '00 Riket', '2581 Pite�', '1480 G�teborg'])].reset_index(drop=True)
slutanvandning = slutanvandning[slutanvandning.region.isin(['0160 T�by', '0117 �ster�ker', '0180 Stockholm', '0980 Gotland', '01 Stockholms l�n', '22 V�sternorrlands l�n', '09 Gotlands l�n', '00 Riket', '2581 Pite�', '1480 G�teborg'])].reset_index(drop=True)
energidata = energidata[energidata.region.isin(['0160 T�by', '0117 �ster�ker', '0180 Stockholm', '0980 Gotland', '01 Stockholms l�n', '22 V�sternorrlands l�n', '09 Gotlands l�n', '00 Riket', '2581 Pite�', '1480 G�teborg'])].reset_index(drop=True)


## Group datasets
leveranser_flytande = leveranser_flytande.groupby(['region', 'ar', 'fornybar_bransletyp'], as_index=False).agg(Sum=('m3', np.sum)).sort_values(by=['Sum']).reset_index(drop=True)
elproduction_bransleanvandning_grouped = elproduction_bransleanvandning.groupby(['region', 'ar', 'fornybar_bransletyp', 'fornybar_produktionssatt'], as_index=False).agg(Sum=('MWh', np.sum)).sort_values(by=['Sum']).reset_index(drop=True)
fjarrvarmeproduktion = fjarrvarmeproduktion.groupby(['region', 'ar', 'fornybar_bransletyp', 'fornybar_produktionssatt'], as_index=False).agg(Sum=('MWh', np.sum)).sort_values(by=['Sum']).reset_index(drop=True)
slutanvandning = slutanvandning.groupby(['region', 'ar', 'fornybar_bransletyp', 'forbrukarkategori'], as_index=False).agg(Sum=('MWh', np.sum)).sort_values(by=['Sum']).reset_index(drop=True)
bransleforbrukning = bransleforbrukning.groupby(['ar', 'fornybar_bransletyp', 'produktionsslag'], as_index=False).agg( Sum=('branslefarbrukning', np.sum)).sort_values(by=['Sum']).reset_index(drop=True)
energidata = energidata.groupby(['region', 'ar', 'fornybar_energityp', 'fornybar_kategori'], as_index=False).agg(Sum=('MWh', np.sum)).sort_values(by=['Sum']).reset_index(drop=True)


leveranser_flytande.ar = pd.to_datetime(leveranser_flytande.ar, format='%Y-%m-%d').dt.year
elproduction_bransleanvandning_grouped.ar = pd.to_datetime(elproduction_bransleanvandning_grouped.ar, format='%Y-%m-%d').dt.year
fjarrvarmeproduktion.ar = pd.to_datetime(fjarrvarmeproduktion.ar, format='%Y-%m-%d').dt.year
slutanvandning.ar = pd.to_datetime(slutanvandning.ar, format='%Y-%m-%d').dt.year
bransleforbrukning.ar = pd.to_datetime(bransleforbrukning.ar, format='%Y-%m-%d').dt.year
energidata.ar = pd.to_datetime(energidata.ar, format='%Y-%m-%d').dt.year



C:\Users\markella.nikolopoulo\AppData\Local\Temp\ipykernel_5704\3475572669.py:4: DtypeWarning: Columns (5,6) have mixed types. Specify dtype option on import or set low_memory=False.
  slutanvandning = pd.read_table('../data/to-use/slutanvandning-region-forbrukarkategori-bransletyp-kategori-energityp-ar.csv', sep=',')


In [111]:
# Constants

AR = 'ar'
M3 = 'm3'
MWH = 'MWh'

BRANSLETYP = 'bransletyp'
FARBRUKARKATEGORI = 'farbrukarkategori'
PRODUCTIONSSATT = 'produktionssatt'
KATEGORI = 'kategori'
ENERGITYP = 'energityp'
PRODUCTIONSSLAG = 'produktionsslag'
BRANSLEFORBRUKNING = 'bransleforbukning'

REGION = 'region'

FORNYBAR_BRANSLETYP = 'fornybar_bransletyp'
FORNYBAR_PRODUCTIONSSATT = 'fornybar_produktionssatt'
FORNYBAR_PRODUCTIONSSLAG = 'fornybar_produktionssatt'
FORNYBAR_ENERGITYP = 'fornybar_energityp'
FORNYBAR_KATEGORI = 'fornybar_kategori'


In [112]:
encodeList = [REGION, FORNYBAR_BRANSLETYP, FORNYBAR_PRODUCTIONSSATT, FORNYBAR_PRODUCTIONSSLAG, FORNYBAR_ENERGITYP, FORNYBAR_KATEGORI]

In [113]:
for encode_item in encodeList:
    if encode_item in leveranser_flytande.columns:
        leveranser_flytande[encode_item].replace(leveranser_flytande[encode_item].unique(), range(
            0, len(leveranser_flytande[encode_item].unique())), inplace=True)

In [114]:
# # split our data - by loading X and y we get a frame of all our patient attributes (age, sex serum etc) and a seperate frame of outcomes
# (
#     attributes_train,
#     attributes_test,
#     progression_train,
#     progression_test,
# ) = train_test_split(leveranser_flytande, leveranser_flytande.Sum, test_size=0.33)

# # Instantiate
# linear_regression = LinearRegression()

# # Fit
# linear_regression.fit(attributes_train, progression_train)

# # Predict
# progression_predict = linear_regression.predict(attributes_test)

for each of the dataframes (not a level of the nexted loop; manualy copied block):

for each region:   
    for each type of energy:   
        for each type of energy (if needed):    
            find the max of ar    
            create series with range ar.max+1:a.max+6   
            create a series with ar as index and Sum as value   
            plot and fit the forcaters   

In [130]:
leveranser_flytande.columns

Index(['region', 'ar', 'fornybar_bransletyp', 'Sum'], dtype='object')

In [127]:
# leveranser_flytande

for region in leveranser_flytande.region.drop_duplicates(keep='first').reset_index(drop=True):
    print(region)
    data = leveranser_flytande[leveranser_flytande.region == region].reset_index(drop=True)    
    for fornybar_bransletyp in data.fornybar_bransletyp.drop_duplicates(keep='first').reset_index(drop=True):
        print(fornybar_bransletyp)
        data = data[data.fornybar_bransletyp == fornybar_bransletyp].reset_index(drop=True)

# elproduction_bransleanvandning_grouped


# fjarrvarmeproduktion


# slutanvandning


# bransleforbrukning


# energidata




AttributeError: 'DataFrame' object has no attribute 'REGION'

In [116]:
leveranser_flytande.ar.max()

1970

In [122]:
leveranser_flytande_s = leveranser_flytande.set_index(['ar', 'region', 'fornybar_bransletyp']).squeeze()

In [124]:
# Get our airline data:
y = leveranser_flytande_s
plot_series(y, labels=['y'])

# Instantiate:
forecaster = NaiveForecaster(strategy='mean')

# Fit:
forecaster.fit(leveranser_flytande)

TypeError: y must be in an sktime compatible format, of scitype Series, Panel or Hierarchical, for instance a pandas.DataFrame with sktime compatible time indices, or with MultiIndex and last(-1) level an sktime compatible time index. See the forecasting tutorial examples/01_forecasting.ipynb, or the data format tutorial examples/AA_datatypes_and_datasets.ipynb,If you think y is already in an sktime supported input format, run sktime.datatypes.check_raise(y, mtype) to diagnose the error, where mtype is the string of the type specification you want for y. Possible mtype specification strings are as follows. "For Hierarchical scitype: ['pd_multiindex_hier']. 

In [82]:

# An sktime ForecastingHorizon object:
sktime_fh = ForecastingHorizon(
    pd.PeriodIndex(pd.date_range("1970", periods=10, freq="Y")), is_relative=False
)

In [79]:
# Predict:
y_pred = forecaster.predict(fh=[1971, 1972, 1973])

# Check it out:
# plot_series(leveranser_flytande, y_pred, labels=["y", "y_pred"])

In [83]:
# Instantiate:
forecaster = NaiveForecaster(sp=12)

# Fit:
forecaster.fit(leveranser_flytande)

# Predict:
y_pred = forecaster.predict(fh=[1971, 1972, 1973])

# Check it out:
# plot_series(leveranser_flytande, y_pred, labels=["y", "y_pred"])

In [84]:

# Instantiate:
forecaster = ThetaForecaster(sp=12)

# Fit:
forecaster.fit(leveranser_flytande)

# Predict:
y_pred_theta = forecaster.predict(fh=[1971, 1972, 1973])

# Check it out:
# plot_series(leveranser_flytande, y_pred, y_pred_theta, labels=["y", "y_pred_naive", "y_pred_theta"])

ValueError: Multiplicative seasonality is not appropriate for zero and negative values

In [85]:
from sktime.forecasting.arima import AutoARIMA

forecaster = AutoARIMA(sp=12, d=0, max_p=2, max_q=2, suppress_warnings=True)
forecaster.fit(leveranser_flytande)

y_pred = forecaster.predict(fh=[1971, 1972, 1973])

c:\ProgramData\Anaconda3\envs\thisone\lib\site-packages\pmdarima\arima\auto.py:444: UserWarning: Input time-series is completely constant; returning a (0, 0, 0) ARMA.
  warnings.warn('Input time-series is completely constant; '


In [87]:
from sktime.forecasting.ets import AutoETS

forecaster = AutoETS(auto=True, n_jobs=-1, sp=12)
forecaster.fit(leveranser_flytande)

y_pred = forecaster.predict(fh=[1971, 1972, 1973])

c:\ProgramData\Anaconda3\envs\thisone\lib\site-packages\sktime\forecasting\ets.py:251: UserWarning: Warning: time series is not strictly positive, multiplicative components are ommitted
  warnings.warn(
